In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, SMOL_LM_135M, SFT_MATH_CONFIG

cfg = SFT_MATH_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 1000

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()

2025-03-22 11:45:56.763 | INFO     | trl_wrapper.trainer_wrapper:init_model:341 - Using device: mps
2025-03-22 11:45:56.763 | INFO     | trl_wrapper.trainer_wrapper:init_model:355 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa
2025-03-22 11:45:57.544 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:376 - Using dataset path: openai/gsm8k
2025-03-22 11:45:57.544 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:381 - Using chat template override: smollmv2
2025-03-22 11:45:57.544 | INFO     | model.utils:__init__:119 - Cache dir: ../dataset_caches/g_s_m8_k_data_module


In [3]:
wrapper.data_module

In [5]:
wrapper.init_trainer()

2025-03-22 11:46:57.168 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:423 - Saving output to: ./runs/03-22-11-46-106301-smollm2-135m-instruct-
2025-03-22 11:46:57.169 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:446 - Initializing trainer, run_name: 03-22-11-46-106301-smollm2-135m-instruct-, wandb project: qwen-math-sft
/Users/brianfitzgerald/Documents/GitHub/smolmodels/notebooks/../trl_wrapper/trainer_wrapper.py:517: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomSFTTrainer.__init__`. Use `processing_class` instead.
  self.trainer = CustomSFTTrainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.STEPS but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

In [ ]:
wrapper.train()